# Working with Parameterized Affinity Functions

In this notebook, we'll demonstrate how to define parameterized affinity functions and perform coordinate-ascent maximum-likelihoood inference to estimate their parameters. There are a few important limitations to keep in mind when doing this. 

1. As currently implemented, the parameter estimate step of coordinate ascent can be quite slow on graphs with more than a few hundred nodes. 
2. Additionally, because the node clustering step of coordinate ascent is implemented via Louvain, only affinity functions that encourage assortative cluster structure are likely to lead to interpretable results. 

Because of these limitations, general parameterized affinity functions are not used in the paper  

> Chodrow, Philip S., Nate Veldt, and Austin R. Benson. "Generative hypergraph clustering: from blockmodels to modularity." arXiv preprint arXiv:2101.09611 (2021).

With this in mind, this feature should be regarded as experimental. 

In [1]:
using Revise
using Pkg; Pkg.activate("../.")
using HyperModularity

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /home/phil/.julia/packages/Revise/BqeJF/src/Revise.jl:1328
 Activating environment at `~/HyperModularity/Project.toml`
┌ Info: Precompiling HyperModularity [bc4352e7-2825-48c1-aa5c-95462f20dcc0]
└ @ Base loading.jl:1260


Mathematically, an affinity function $\Omega$ is defined by a function $\omega(p, \alpha)$, where $p$ is a partition vector and $\alpha$ is a vector of parameters. It is also necessary to specify the largest hyperedge size under consideration. Here's an example:

In [2]:
n = 100
kmax = 4

function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

Ω = partitionIntensityFunction(ω, kmax);

### Generating Synthetic Hypergraphs

Once an affinity function has been constructed, it may be used to sample from the DCHSBM model. It is necessary to also set a node cluster vector, a vector of degree parameters $\vartheta$, and a value of the parameter $\alpha$ passed to $\Omega$. 

In [3]:
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)
α = vcat(repeat([5.0], kmax), 0.2*(1:kmax))

H = sampleSBM(Z, ϑ, Ω;α=α, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [6, 4, 10, 11, 11, 5, 4, 11, 11, 8  …  3, 5, 3, 3, 5, 5, 8, 3, 6, 11]


In [4]:
Q = round(Float64(modularity(H, Z, Ω;α = α)), digits = 2)
println("The modularity of the generated hypergraph with specified parameters is $Q.")

The modularity of the generated hypergraph with specified parameters is -20604.06.


### Unknown $\alpha$

In the inference problem, we assume that we do not know either the cluster labeling or the parameter vector $\alpha$ (or both), and attempt to learn them from data. In this section, we'll show how to estimate the parameter $\alpha$ from a known cluster labeling. 

To reiterate the limitations above, at the current time, this should only be expected to work well on relatively small hypergraphs with assortative affinity functions. 

#### Important Note

In general, due to the normalization condition imposed on the affinity function $\Omega$ and degree parameters $\vartheta$ when performing inference, both the modularities and parameter estimates so obtained may vary considerably from the values obtained when generating the model. More specifically: 

- You won't get back the original $\alpha$ even approximately (unless you custom-engineered $\Omega$ to satisfy the normalization conditions). 
- The optimized modularity will not resemble the modularity calculated with the original $\alpha$ (unless you custom-engineered $\Omega$ to satisfy the normalization conditions). 

This issue reflects the fact that we impose a constraint during inference that is not imposed when generating the hypergraph. 

In [5]:
α̂ = learnParameters(H, Z, Ω, α; max_iters = 100, verbose = false, tol = 1e-8)

8-element Array{Float64,1}:
 5.0
 4.590113950118679
 4.55105489111123
 3.6144204657667953
 0.5109279534173448
 0.7478582083900334
 0.9454931654997941
 1.1718251467391303

In [6]:
Q = round(Float64(modularity(H, Z, Ω;α = α̂)), digits = 2)
println("The modularity of the generated hypergraph with estimated parameter α̂ is $Q.")

The modularity of the generated hypergraph with estimated parameter α̂ is -3113.04.


### Unknown Partition

In most practical contexts, we do not know the labeling either. In this case, we can alternate between estimates of $\hat{\alpha}$ using `learnParameters` and estimates of the cluster labels $\mathbf{Z}$ using one of the supplied Louvain algorithms. 

In [7]:
α̂ = vcat(repeat([1.0], kmax), 0.1*(1:kmax)) # initial guess
Ẑ = ones(n)

n_rounds = 10

for i ∈ 1:n_rounds
    Ẑ = SuperNodeLouvain(H, Ω; α = α̂, verbose = false, )
    α̂ = learnParameters(H, Ẑ, Ω, α̂; tol = 1e-8)
    
    Q = round(Float64(modularity(H, Ẑ, Ω;α = α̂)), digits = 2)
    k = length(unique(Ẑ))
    println("Current modularity $Q with $k clusters.")
end

Current modularity -3286.79 with 100 clusters.
Current modularity -3087.39 with 10 clusters.
Current modularity -3078.78 with 10 clusters.
Current modularity -3075.98 with 11 clusters.
Current modularity -3074.54 with 11 clusters.
Current modularity -3071.64 with 11 clusters.
Current modularity -3072.31 with 12 clusters.
Current modularity -3075.57 with 11 clusters.
Current modularity -3071.17 with 11 clusters.
Current modularity -3072.49 with 11 clusters.


We recovered a clustering of 16 clusters, when the true data set contained only 5. Despite this, our clustering is noticeably correlated with the true clusters as measured by the mutual information. 

In [8]:
mutualInformation(Z, Ẑ, true)

0.5610321374864219